In [2]:
import random
import torch
import random
import torch
import pickle
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle

# !pip install tensorflow

In [3]:
#load data from kaggle

import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
import kagglehub
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from transformers import get_scheduler

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# Load the dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv")

# Check the first few rows of the dataset
print(df.head())

100%|██████████| 92.1M/92.1M [00:01<00:00, 79.7MB/s]

Extracting files...


   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  


In [ ]:
#check what's missing in the data

# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Check how many rows have missing data
rows_with_missing_values = df[df.isnull().any(axis=1)]

# Print out the missing values summary and rows with missing data
print("Missing values per column:\n", missing_values)
print("\nNumber of rows with missing values:", rows_with_missing_values.shape[0])

# Preview the rows with missing values
print("\nRows with missing values:")
print(rows_with_missing_values.head())

Missing values per column:
 Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

Number of rows with missing values: 597

Rows with missing values:
     Unnamed: 0 title                                               text  \
1             1   NaN     Did they post their votes for Hillary already?   
43           43   NaN  True. Hillary needs a distraction and what bet...   
162         162   NaN  All eyes on Electoral delegates. The People kn...   
185         185   NaN                                               Cool   
269         269   NaN  A leading US senator: US Supporting War in Syr...   

     label  
1        1  
43       1  
162      1  
185      1  
269      1  


In [10]:
#clean the data of missing values

# Ensure that all values in 'text' are strings
df['text'] = df['text'].fillna('')  # Fill NaN values with an empty string
texts = df['text'].astype(str).tolist()  # Convert to list of strings

# # map labels to binary values
# df['label'] = df['label'].map({'true': 1, 'false': 0})

# Check the first few entries
print(texts[:1])

print(len(texts[0]))

# Split the dataset
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['text'], df['label'], test_size=0.2, random_state=42

dataset = df[['text', 'label']]
print(dataset.head())
dataset = shuffle(dataset, random_state=42).reset_index(drop=True)
texts = dataset['text'].tolist()
labels = dataset['label'].tolist()
print('\n', texts[:2], '\n', labels[:2])

count, sec, thr, you = 0, 0, 0, 0
new_dic = []
for i, k in enumerate(texts):
  if len(k) < 200:
    thr += 1
    # if len(k) > 100:
    #   print(k, labels[i])
  if k == ' ':
    count += 1

  if k[:4] == 'http':
    you += 1

  if k not in new_dic:
    new_dic.append(k)

new_data = []
for i in texts:
  if len(i) > 50 and i not in new_data:
    new_data.append(i)

print(len(new_data))
print(count, sec, thr, you, len(texts), len(new_dic))

['No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred t

In [ ]:
)#tokenize with bert
#mattias says to put this into batches (XXX - ?)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text, adjust the max_length parameter based on data analysis (XXX)
def encode_text(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Tokenize the text data
#train_encodings = encode_text(texts[:10])
texts_encodings = tokenizer(list(texts), padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert labels to tensors
labels = torch.tensor(list(labels))

# Check the tokenized data
print(texts_encodings)
print(len(texts))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'input_ids': tensor([[  101,   102,     0,  ...,     0,     0,     0],
        [  101,  1998,  2085,  ...,     0,     0,     0],
        [  101, 11093,  1010,  ...,  4295,  1012,   102],
        ...,
        [  101,  1006, 26665,  ...,     0,     0,     0],
        [  101,  1037,  2082,  ...,  2005,  7987,   102],
        [  101,  2023,  2028,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}
72134


In [ ]:
# with open("temp.pkl", "rb") as fOut:
#   lol = pickle.load(fOut)
# # print(lol['sentences'])

In [ ]:
# print(len(lol['sentences']))
# # print(len(lol['sentences'].index))

72134


Unnamed: 0                                                60264
title         Elon Musk’s Tesla Stock Up $2 Billion Since Jo...
text          Although Tesla CEO Elon Musk shocked Silicon V...
label                                                         0
Name: 60264, dtype: object
tensor(0)


In [ ]:
import h5py
import numpy as np

input_ids = texts_encodings['input_ids'].numpy()
attention_mask = texts_encodings['attention_mask'].numpy()
labels = np.array(labels)

with h5py.File('Dataset_first.h5', 'w') as hf:
    hf.create_dataset('input_ids', data=input_ids, compression="gzip")
    hf.create_dataset('attention_mask', data=attention_mask, compression="gzip")
    hf.create_dataset('labels', data=labels, compression="gzip")
    dt = h5py.string_dtype(encoding='utf-8')
    hf.create_dataset('texts', data=texts, dtype=dt)


In [ ]:


# Open the HDF5 file in read mode
with h5py.File('Dataset_first.h5', 'r') as hf:
    # Load the datasets
    input_ids = np.array(hf['input_ids'])
    attention_mask = np.array(hf['attention_mask'])
    labels = np.array(hf['labels'])

# Check the loaded data
print("Input IDs shape:", input_ids.shape)
print("Attention Mask shape:", attention_mask.shape)
print("Labels shape:", labels.shape)

# Example: Access a single data point
print("First input_ids:", input_ids[10])
print("First attention_mask:", attention_mask[10])
print("First label:", labels[10])


Input IDs shape: (72134, 512)
Attention Mask shape: (72134, 512)
Labels shape: (72134,)
First input_ids: [  101  4419  2739  1996  2047  2259  2335  2988  3041  2023  2733  2008
  6521  2371 26608  2044  8398  5496  2032  1997  4487 14540 18232 24228
  2138  2002 28667 13901  2370  2013  1996  3425  2533  1055  4812  2046
  1996  8398  3049  1055  7208  2000  3607  1012  8398 22416  1037  5164
  1997 23862  2012  6521  1010  2040  2052  2101  2175  2006  2000  2360
  8398  1055 16360 20026  5685  2075  2001  1996  2087 28284  2724  2002
  5281  2004  1037  2270  7947  1010  2429  2000  1996  2335  1012 16360
  1012 21510  2638  5380  1010  1040  1011 10250 10128  1012  1010  1056
 28394  3064  5958  2008  4905  2236  5076  6521  2085  4282  2129  3060
  1011  4841  2514  2044  2002  2001  7283 26608  2011  2343  8398  2058
  2010 28667 10383  2140  1999  1996  3607  4812  1012  2006  1037 16110
  1999  2089  1010  5380  3615  2000  6521  2004  2200  4795  1010  2077
  5815  1010  1045 

Hash comparison passed. Data is identical.
